In [4]:
import threading
import tracemalloc
from ortools.linear_solver import pywraplp
tracemalloc.start()
def solve_irp(dist_matrix_data, dist_supplier_data, nb_customers, horizon_length, capacity, start_level_supplier, production_rate_supplier, start_level, demand_rate, max_level, min_level, holding_cost_supplier, holding_cost):
    solver = pywraplp.Solver.CreateSolver('SCIP')
    if not solver:
        return None

    infinity = solver.infinity()
    # Variables
    delivery = [[solver.NumVar(0, capacity, f'delivery[{t}][{i}]') for i in range(nb_customers)]
                for t in range(horizon_length)]

    route = [[[solver.BoolVar(f'route[{t}][{i}][{j}]') for j in range(nb_customers)]
             for i in range(nb_customers)]
             for t in range(horizon_length + 1)]

    # Distance matrix and supplier distances as constants
    dist_matrix = dist_matrix_data
    dist_supplier = dist_supplier_data

    # Inventory at supplier
    inventory_supplier = [solver.NumVar(0, infinity, f'inventory_supplier[{t}]') for t in range(horizon_length + 1)]
    solver.Add(inventory_supplier[0] == start_level_supplier)

    for t in range(1, horizon_length + 1):
        solver.Add(inventory_supplier[t] == inventory_supplier[t - 1] - sum(delivery[t - 1][i] for i in range(nb_customers)) + production_rate_supplier)
        if t != horizon_length:
            solver.Add(inventory_supplier[t] >= sum(delivery[t][i] for i in range(nb_customers)))
        solver.Add(inventory_supplier[t] >= 0)
        solver.Add(delivery[t-1][0] == 0)

    # Inventory at customers
    inventory = [[solver.NumVar(0, infinity, f'inventory[{i}][{t}]') for t in range(horizon_length + 1)] for i in range(nb_customers)]
    for i in range(1, nb_customers):
        solver.Add(inventory[i][0] == start_level[i-1])
        for t in range(1, horizon_length + 1):
            solver.Add(inventory[i][t] == inventory[i][t - 1] + delivery[t - 1][i] - demand_rate[i-1])
            solver.Add(inventory[i][t] >= min_level[i-1])

    # Capacity constraints
    for t in range(horizon_length):
        solver.Add(sum(delivery[t][i] for i in range(nb_customers)) <= capacity)
        # Maximum level constraints
        for i in range(1, nb_customers):
            solver.Add(delivery[t][i] <= max_level[i-1] - inventory[i][t])
            solver.Add(delivery[t][i] <= max_level[i-1] * sum(route[t][i][j] for j in range(nb_customers)))

    # Total costs
    total_cost_inventory_supplier = holding_cost_supplier * sum(inventory_supplier[t] for t in range(1, horizon_length + 1))
    total_cost_inventory = sum(holding_cost[i-1] * sum(inventory[i][t] for t in range(1, horizon_length + 1)) for i in range(nb_customers))

    # Distance traveled at each time instant
    for t in range(horizon_length):
        for i in range(1, nb_customers):
            solver.Add(sum(route[t][i][j] for j in range(nb_customers)) == 1)
            solver.Add(sum(route[t][j][i] for j in range(nb_customers)) == 1)

        solver.Add(sum(route[t][0][j] for j in range(1, nb_customers)) == 1)  # Must leave supplier
        solver.Add(sum(route[t][i][0] for i in range(1, nb_customers)) == 1)  # Must return to supplier

    # Linking delivery and routes
    total_cost_route = 0
    for t in range(horizon_length):
        for i in range(nb_customers):
            solver.Add(delivery[t][i] <= capacity * sum(route[t][i][j] for j in range(nb_customers)))
    for t in range(horizon_length):
        for i in range(nb_customers):
            for j in range(nb_customers):
                total_cost_route += route[t][i][j] * dist_matrix[i-1][j-1]
        for i in range(1, nb_customers):
            total_cost_route += route[t][0][i] * dist_supplier[i-1]
            total_cost_route += route[t][i][0] * dist_supplier[i-1]

    # Objective: minimize the sum of all costs
    objective = total_cost_inventory_supplier + total_cost_inventory + total_cost_route
    solver.Minimize(objective)

    # Solver parameters
    # No need to set time limit here, we'll use thread to enforce timeout

    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        print('Solution:')
        print('Objective value =', solver.Objective().Value())
    elif status == pywraplp.Solver.FEASIBLE:
        print('A potentially suboptimal solution was found:')
        print('Objective value =', solver.Objective().Value())
    else:
        print('The solver did not find an optimal solution.')
        return None

import numpy as np

def read_input(filename):
    with open(filename, "r") as file_it:
        lines = file_it.readline().strip().split()
        nb_customers = int(lines[0])
        horizon_length = int(lines[1])
        capacity = int(lines[2])
        nb_vehicles = int(lines[3])
        start_level = [None] * (nb_customers-1)
        max_level = [None] * (nb_customers-1)
        min_level = [None] * (nb_customers-1)
        demand_rate = [None] * (nb_customers-1)
        holding_cost = [None] * (nb_customers-1)
        lines = file_it.readline().strip().split()
        start_level_supplier = int(lines[1])
        production_rate_supplier = int(lines[2])
        holding_cost_supplier = float((lines[3]))
        for i in range(nb_customers-1):
            lines = file_it.readline().strip().split()
            start_level[i] = int(lines[1])
            max_level[i] = int(lines[2])
            min_level[i] = int(lines[3])
            demand_rate[i] = int(lines[4])
            holding_cost[i] = float(lines[5])
        lines = file_it.readline().strip().split()
        dist_supplier_data = [None] * (nb_customers-1)
        for i in range(nb_customers-1):
            dist_supplier_data[i] = int(float(lines[i+1]))
        dist_matrix_data = [[0 for _ in range(nb_customers-1)] for _ in range(nb_customers-1)]
        for i in range(nb_customers-1):
            lines = file_it.readline().strip().split()
            for j in range(nb_customers-1):
                dist_matrix_data[i][j] = int(float(lines[j+1]))
    return (dist_matrix_data, dist_supplier_data, nb_customers, horizon_length, capacity*nb_customers, start_level_supplier, production_rate_supplier, start_level, demand_rate, max_level, min_level, holding_cost_supplier, holding_cost)

# Read input data
input_data = read_input("test.txt")
# Create a thread to run the solve_irp function
solver_thread = threading.Thread(target=solve_irp, args=input_data)

# Start the thread
solver_thread.start()

# Set the timeout duration in seconds
timeout_duration = 900

# Wait for the thread to finish or timeout
solver_thread.join(timeout_duration)

if solver_thread.is_alive():
    print("No Solution found")
    print("The solver timed out after {} seconds.".format(timeout_duration))
else:
    print("Solver finished within the time limit.")
current, peak = tracemalloc.get_traced_memory()
print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")
tracemalloc.stop()

Solution:
Objective value = 1216.01
Solver finished within the time limit.
Current memory usage is 105.672234MB; Peak was 106.119374MB
